In [43]:
import torch
from geqtrain.nn.nonlinearities import ShiftedSoftPlus, ShiftedSoftPlusModule, SwiGLUModule, SwiGLU
from geqtrain.nn._fc import select_nonlinearity
from e3nn.math import normalize2mom
import random


In [58]:
@torch.no_grad()
def test_gain(non_linearity:str, gain:float=None, use_normalize2mom:bool=False, seed: int=42):
  # todo add selection of non linearity via method that is in
  activation = select_nonlinearity(non_linearity)
  in_size = 1000
  out_size = in_size if non_linearity != "swiglu" else 2*in_size

  if not gain and not use_normalize2mom: raise ValueError('no gain (nor automatic gain) provided, idk what to do')
  if gain and use_normalize2mom: raise ValueError('both gain and automatic gain provided, idk what to do')

  # todo implement automatic gain
  # nonlinearity = {
  #   None: None,
  #   "silu": torch.nn.functional.silu,
  #   "ssp": ShiftedSoftPlusModule,
  #   "selu": torch.nn.functional.selu,
  #   "relu": torch.nn.functional.relu,
  #   "swiglu": SwiGLUModule,
  # }[non_linearity]
  # if use_normalize2mom: gain = normalize2mom()


  for test in range(5):
    seed = random.randint(0, 100000)
    torch.manual_seed(seed)
    a = torch.randn(in_size, in_size)
    l = torch.nn.Linear(in_size, out_size, bias=False)
    torch.nn.init.orthogonal_(l.weight, gain=gain)
    for i in range(100):
      x = activation(l(a))
    print (f"Test {test} - input std: {a.std().item():.6f}, out std: {x.std().item():.6f}, difference: {a.std().item() - x.std().item()}")
    if a.std().item() - x.std().item() > 0:
      print("Increase gain")
    else: print("Decrease gain")

In [60]:
test_gain('swiglu', gain=1.79186)
# gain=1.8, N = 50000
# Test 0 - input std: 1.0000, out std: 1.0097
# Test 1 - input std: 0.9993, out std: 1.0040
# Test 2 - input std: 1.0000, out std: 1.0099

# gain=1.78, N = 50000
# Test 0 - input std: 1.0000, out std: 0.9855
# Test 1 - input std: 0.9993, out std: 0.9800
# Test 2 - input std: 1.0000, out std: 0.9858

# gain=1.78, N = 100
# Test 0 - input std: 1.0000, out std: 0.9855
# Test 1 - input std: 0.9993, out std: 0.9800
# Test 2 - input std: 1.0000, out std: 0.9858

Test 0 - input std: 1.000839, out std: 0.997611, difference: 0.0032287240028381348
Increase gain
Test 1 - input std: 1.000109, out std: 0.999631, difference: 0.00047779083251953125
Increase gain
Test 2 - input std: 0.999263, out std: 0.994219, difference: 0.005044043064117432
Increase gain
Test 3 - input std: 0.999107, out std: 0.998230, difference: 0.0008775591850280762
Increase gain
Test 4 - input std: 1.001136, out std: 1.002058, difference: -0.0009222030639648438
Decrease gain
